In [2]:
from xml.etree import ElementTree as ET
from lxml import etree as ET
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [36]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [83]:
# print names of all countries and their cities
for element in document_tree.iter('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


In [5]:
document = ET.parse( './data/mondial_database.xml' )
ims = pd.DataFrame()
n = []
im = []
for country in document.xpath('//country'):
    if country.find('infant_mortality') is None:
        continue
    n.append(country.find('name').text)
    im.append(country.find('infant_mortality').text)
ims['country'] = n
ims['im'] = im
ims = ims.sort_values('im', ascending=False)
ims.head()


,country,im
197,Central African Republic,92.86
214,Guinea-Bissau,90.92
198,Chad,90.3
159,Argentina,9.96
66,Thailand,9.86


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )
ims = pd.DataFrame()
n = []
im = []
for country in document.iterfind('country'):
    if country.find('infant_mortality') is None:
        continue
    n.append(country.find('name').text)
    im.append(country.find('infant_mortality').text)
ims['country'] = n
im = pd.to_numeric(im)
ims['im'] = im
ims.sort_values('im', ascending=True).head(10)


,country,im
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [17]:
#10 cities with the largest population
document = ET.parse( './data/mondial_database.xml' )
document = document.getroot()
ims = pd.DataFrame()
n = []
pop = []
for city in document.findall('.//city'):
    if city.find('population') is None:
        continue
    n.append(city.find('name').text)
    pop.append(city.findall('population')[-1].text)
ims['City'] = n
pop = pd.to_numeric(pop, errors='ignore')
ims['population'] = pop
ims.sort_values('population', ascending=False).head(10)
#Using absolute paths to get cities so some of there are missing. They are at different levels within the xml sheet


,City,population
1251,Shanghai,22315474
707,Istanbul,13710512
1421,Mumbai,12442373
443,Moskva,11979529
1250,Beijing,11716620
2594,São Paulo,11152344
1252,Tianjin,11090314
974,Guangzhou,11071424
1467,Delhi,11034555
977,Shenzhen,10358381


In [74]:
#10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

#
document = ET.parse( './data/mondial_database.xml' )
egp = pd.DataFrame()
for country in document.xpath('//country'):
    for e in country.findall('ethnicgroup'):
        egp = egp.append([[float(e.attrib['percentage']), e.text, e.getparent().findall('population')[-1].text]])
ep['ethnicicty'] = eg
egp.columns = ['percent', 'ethnicity', 'population']
egp = egp.sort_values(['ethnicity'])
egp['population'] = pd.to_numeric(egp['population'])
egp['pop_of_ethnic_group'] = (egp['percent'] / 100) * egp['population'] 
egp = egp.groupby('ethnicity').sum()
egp[['pop_of_ethnic_group']].sort_values('pop_of_ethnic_group', ascending=False).head(10)

,pop_of_ethnic_group
ethnicity,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [75]:
#name and country of a) longest river, b) 
document = ET.parse( './data/mondial_database.xml' )
csn = []
csc = []
rivers = pd.DataFrame()
rn = []
l = []
cc = []
#Replace country code with Country name
for country in document.iterfind('country'):
    csn.append(country.find('name').text)
    csc.append(country.attrib['car_code'])

for river in document.iterfind('river'):
    if river.find('length') is None:
        continue
    r = str()
    rn.append(river.find('name').text)
    l.append(river.find('length').text)
    cc.append(river.find('source').attrib['country'])

l = pd.to_numeric(l)
rivers['country'] = cc
rivers['name'] = rn
rivers['length'] = l
rivers = rivers.sort_values(['length'], ascending=False)
rivers['country'] = rivers['country'].replace(csc, csn)
rivers.head(1)


,country,name,length
174,Peru,Amazonas,6448.0


In [90]:
#largest lake
lakes = pd.DataFrame()
csn = []
csc = []
ln = []
a = []
cc = []
#Replace country code with Country name
for country in document.iterfind('country'):
    csn.append(country.find('name').text)
    csc.append(country.attrib['car_code'])

for lake in document.iterfind('lake'):
    if lake.find('name') is None:
        ln.append('')
    else:
        ln.append(lake.find('name').text)
    if lake.find('area') is None:
        a.append(0)
    else:
        a.append(lake.find('area').text)
    if lake.find('located') is None:
        cc.append(lake.attrib['country'])
    else:
        cc.append(lake.find('located').attrib['country'])
a = pd.to_numeric(a)
lakes['country'] = cc
lakes['name'] = ln
lakes['area'] = a
lakes = lakes.sort_values(['area'], ascending=False)
lakes['country'] = lakes['country'].replace(csc, csn)
lakes.head(5)


,country,name,area
54,Russia,Caspian Sea,386400.0
109,Canada,Lake Superior,82103.0
81,Tanzania,Lake Victoria,68870.0
106,Canada,Lake Huron,59600.0
108,United States,Lake Michigan,57800.0


In [106]:
#airport at highest elevation
airport = pd.DataFrame()
csn = []
csc = []
an = []
elev = []
cc = []
#Replace country code with Country name
for country in document.iterfind('country'):
    csn.append(country.find('name').text)
    csc.append(country.attrib['car_code'])

for ap in document.iterfind('airport'):
    if ap.find('name') is None:
        an.append('')
    else:
        an.append(ap.find('name').text)
    if ap.find('elevation') is None:
        elev.append(0)
    else:
        elev.append(ap.find('elevation').text)
    cc.append(ap.attrib['country'])
elev = pd.to_numeric(elev)
airport['country'] = cc
airport['name'] = an
airport['elevation'] = elev
airport['country'] = airport['country'].replace(csc, csn)
airport = airport.sort_values(['elevation'], ascending=False)
airport.head(5)

,country,name,elevation
80,Bolivia,El Alto Intl,4063.0
219,China,Lhasa-Gonggar,4005.0
241,China,Yushu Batang,3963.0
813,Peru,Juliaca,3827.0
815,Peru,Teniente Alejandro Velasco Astete Intl,3311.0
